# NEWS SCRAPER MINI PROJECT - DATA PREPARATION

> This notebook focuses solely on preparing scraped news article data for the Sentiment Analysis project requirements.
>
> The content of this notebook will consists of 3 parts:
>
> - **1. IMPORT LIBRARIES** - Importing the necessary libraries.
> - **2. OVERVIEW** - Conducting initial data sanity checks (info, null values, etc.).
> - **3. DATA CLEANING** - Carrying out the main data cleansing process, from checking the data type to adjusting the text to a stemmed format.
>
> In summary, this notebook serves as a crucial step in the data preparation process for this mini project. By importing the necessary libraries, conducting an initial data overview, and performing data cleaning, we can ensure that the data is in a suitable format for further process.

## IMPORT LIBRARIES

In [1]:
# data wrangling
import numpy as np
import pandas as pd
import string
import re
import html2text
from tqdm import tqdm
from bs4 import BeautifulSoup

# nlp wrangling
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer

# warning
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='ignore', category=FutureWarning)

## OVERVIEW
> After importing the necessary libraries, we will check the initial data state, summary and general data overview.\
> In this section, we will breakdown the data and try to see whether the data is clean enough to move to the next process.

In [2]:
# load data
df = pd.read_csv("data/kompas_news_article.csv")

In [3]:
# show head
df.head()

,author,publish_date,news_article_raw,news_article_semi
0,Tito Hilmawan Reditya,"Kompas.com - 26/04/2023, 18:15 WIB","[<p><strong>LONDON, KOMPAS.com</strong> - Inva...","LONDON, KOMPAS.com - Invasi China ke Taiwan di..."
1,Aditya Jaya Iswara,"Kompas.com - 26/04/2023, 17:35 WIB","[<p><strong>BANGKOK, KOMPAS.com</strong> - Pol...","BANGKOK, KOMPAS.com - Polisi Thailand pada Sel..."
2,Danur Lambang Pristiandaru,"Kompas.com - 26/04/2023, 16:01 WIB","[<p><strong></strong></p>, <p><strong>PRETORIA...","PRETORIA, KOMPAS.com – Presiden Afrika Selatan..."
3,Aditya Jaya Iswara,"Kompas.com - 26/04/2023, 15:34 WIB","[<p><strong>WELLINGTON, KOMPAS.com</strong> - ...","WELLINGTON, KOMPAS.com - Mantan Perdana Menter..."
4,Aditya Jaya Iswara,"Kompas.com - 26/04/2023, 14:48 WIB","[<p><strong>MOSKWA, KOMPAS.com</strong> - <a c...","MOSKWA, KOMPAS.com - Rusia mengganti KFC yang ..."


In [4]:
# show info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   author             415 non-null    object
 1   publish_date       415 non-null    object
 2   news_article_raw   415 non-null    object
 3   news_article_semi  415 non-null    object
dtypes: object(4)
memory usage: 13.1+ KB


In [5]:
# check null
df.isna().sum()

author               0
publish_date         0
news_article_raw     0
news_article_semi    0
dtype: int64

> Seems that the data is already 100% proper for the next steps.

## DATA CLEANING

> In this section we will clean the data, so that it may fit the purpose of the project itself (sentiment analysis). \
> Meaning that the news article column  must "fit" the nlp project requirements, in this case we will try to get the article to a *stemmed format*.

> Therefore, the data cleaning will be separated into several steps:
> 1. Adjusting **`publish_date`** column to a **`pd.datetime`** format.
> 
> 2. Manipulating the string in the **`news_article_raw`** column using **`regular expression`**, **`html2text`** library, and other string manipulation methods.
>
> 3. Using basic nlp disciplines *(casefolding, tokeninzing, stopword removal)* to prepare the string to a proper stemmed format.
>
> 4. Lastly export all dataframe to a csv format for the next step.

In [6]:
# adjust publish date to datetime format
pub_fix = []
for i in df["publish_date"]:
    temp = str(i).strip("Kompas.com - ")
    pub_fix.append(re.sub(" WIB", "", temp))

In [7]:
# add adjusted publish date to dataframe
df["publish_date_adj"] = pub_fix
df["publish_date_adj"] = pd.to_datetime(df["publish_date_adj"])

In [8]:
# show data
df.head()

,author,publish_date,news_article_raw,news_article_semi,publish_date_adj
0,Tito Hilmawan Reditya,"Kompas.com - 26/04/2023, 18:15 WIB","[<p><strong>LONDON, KOMPAS.com</strong> - Inva...","LONDON, KOMPAS.com - Invasi China ke Taiwan di...",2023-04-26 18:15:00
1,Aditya Jaya Iswara,"Kompas.com - 26/04/2023, 17:35 WIB","[<p><strong>BANGKOK, KOMPAS.com</strong> - Pol...","BANGKOK, KOMPAS.com - Polisi Thailand pada Sel...",2023-04-26 17:35:00
2,Danur Lambang Pristiandaru,"Kompas.com - 26/04/2023, 16:01 WIB","[<p><strong></strong></p>, <p><strong>PRETORIA...","PRETORIA, KOMPAS.com – Presiden Afrika Selatan...",2023-04-26 16:01:00
3,Aditya Jaya Iswara,"Kompas.com - 26/04/2023, 15:34 WIB","[<p><strong>WELLINGTON, KOMPAS.com</strong> - ...","WELLINGTON, KOMPAS.com - Mantan Perdana Menter...",2023-04-26 15:34:00
4,Aditya Jaya Iswara,"Kompas.com - 26/04/2023, 14:48 WIB","[<p><strong>MOSKWA, KOMPAS.com</strong> - <a c...","MOSKWA, KOMPAS.com - Rusia mengganti KFC yang ...",2023-04-26 14:48:00


In [9]:
# clean news article raw, examine the pattern first
sample = df["news_article_raw"][0]

In [10]:
# preview and examine pattern
print(BeautifulSoup(sample, 'html.parser').prettify())

[
<p>
 <strong>
  LONDON, KOMPAS.com
 </strong>
 - Invasi China ke Taiwan disebut akan menghancurkan perdagangan dunia.
</p>
,
<p>
 "Jarak tidak akan memberikan perlindungan terhadap pukulan dahsyat yang tak terelakkan terhadap ekonomi global," kata Menteri Luar Negeri Inggris, James Cleverly.
</p>
,
<p>
 Dia memperingatkan dalam pidato tentang hubungan Inggris dengan Beijing.
</p>
,
<p>
 <strong>
  Baca juga:
  <a class="inner-link-baca-juga" href="https://www.kompas.com/global/read/2023/04/25/121500270/perubahan-kebijakan-twitter-picu-lonjakan-propaganda-oleh-rusia-dan-china" target="_self">
   Perubahan Kebijakan Twitter Picu Lonjakan Propaganda oleh Rusia dan China
  </a>
 </strong>
</p>
,
<p>
 Sambutan ini berbeda dari apa yang disampaikan Presiden Perancis Emmanuel Macron untuk menjauhkan Eropa dari potensi keterlibatan AS dalam konflik di masa depan atas Taiwan.
</p>
,
<p>
 Macron dengan tegas mendukung keberlanjutan keterlibatan yang dijaga dengan Beijing.
</p>
,
<p>
 Tapi, dil

In [11]:
# define regex pattern
pattern = r"<p>(.*?)</p>"

# extract string between <p> and </p> tags
sample_text = re.findall(pattern, sample)

# preview the temp result
print(sample_text)

['<strong>LONDON, KOMPAS.com</strong> - Invasi China ke Taiwan disebut akan menghancurkan perdagangan dunia.', '"Jarak tidak akan memberikan perlindungan terhadap pukulan dahsyat yang tak terelakkan terhadap ekonomi global," kata Menteri Luar Negeri Inggris, James Cleverly.', 'Dia memperingatkan dalam pidato tentang hubungan Inggris dengan Beijing.', '<strong>Baca juga: <a class="inner-link-baca-juga" href="https://www.kompas.com/global/read/2023/04/25/121500270/perubahan-kebijakan-twitter-picu-lonjakan-propaganda-oleh-rusia-dan-china" target="_self">Perubahan Kebijakan Twitter Picu Lonjakan Propaganda oleh Rusia dan China</a></strong>', 'Sambutan ini berbeda dari apa yang disampaikan Presiden Perancis Emmanuel Macron untuk menjauhkan Eropa dari potensi keterlibatan AS dalam konflik di masa depan atas Taiwan.', 'Macron dengan tegas mendukung keberlanjutan keterlibatan yang dijaga dengan Beijing.', 'Tapi, dilansir dari <em>Guardian</em>, Cleverly mengatakan bahwa tidak ada negara yang d

In [12]:
# join temp result to a sentence
sample_text = ' '.join(sample_text[:])

# preview the result
print(sample_text)

<strong>LONDON, KOMPAS.com</strong> - Invasi China ke Taiwan disebut akan menghancurkan perdagangan dunia. "Jarak tidak akan memberikan perlindungan terhadap pukulan dahsyat yang tak terelakkan terhadap ekonomi global," kata Menteri Luar Negeri Inggris, James Cleverly. Dia memperingatkan dalam pidato tentang hubungan Inggris dengan Beijing. <strong>Baca juga: <a class="inner-link-baca-juga" href="https://www.kompas.com/global/read/2023/04/25/121500270/perubahan-kebijakan-twitter-picu-lonjakan-propaganda-oleh-rusia-dan-china" target="_self">Perubahan Kebijakan Twitter Picu Lonjakan Propaganda oleh Rusia dan China</a></strong> Sambutan ini berbeda dari apa yang disampaikan Presiden Perancis Emmanuel Macron untuk menjauhkan Eropa dari potensi keterlibatan AS dalam konflik di masa depan atas Taiwan. Macron dengan tegas mendukung keberlanjutan keterlibatan yang dijaga dengan Beijing. Tapi, dilansir dari <em>Guardian</em>, Cleverly mengatakan bahwa tidak ada negara yang dapat melindungi diri

In [13]:
# use html2text to clean html format
text_maker = html2text.HTML2Text()
text_maker.ignore_links = True
text_maker.bypass_tables = True
sample_text = text_maker.handle(sample_text)

# preview html text result
print(sample_text)

**LONDON, KOMPAS.com** \- Invasi China ke Taiwan disebut akan menghancurkan
perdagangan dunia. "Jarak tidak akan memberikan perlindungan terhadap pukulan
dahsyat yang tak terelakkan terhadap ekonomi global," kata Menteri Luar Negeri
Inggris, James Cleverly. Dia memperingatkan dalam pidato tentang hubungan
Inggris dengan Beijing. **Baca juga:Perubahan Kebijakan Twitter Picu Lonjakan
Propaganda oleh Rusia dan China** Sambutan ini berbeda dari apa yang
disampaikan Presiden Perancis Emmanuel Macron untuk menjauhkan Eropa dari
potensi keterlibatan AS dalam konflik di masa depan atas Taiwan. Macron dengan
tegas mendukung keberlanjutan keterlibatan yang dijaga dengan Beijing. Tapi,
dilansir dari _Guardian_ , Cleverly mengatakan bahwa tidak ada negara yang
dapat melindungi diri dari dampak perang di Taiwan. Dia menambahkan bahwa dia
ngeri memikirkan kehancuran finansial dan manusia yang akan terjadi. Mendesak
tidak ada pihak yang mengambil tindakan sepihak untuk mengubah status quo, dia
menega

In [14]:
# remove text enclosed in square brackets, asterisks or underscores
sample_text = re.sub(r'\[.*?\]', '', sample_text)
sample_text = re.sub(r'(\*|_).*?(\*|_)', '', sample_text).replace(" \- ", " ")

In [15]:
# preview final result
print(sample_text)

LONDON, KOMPAS.com Invasi China ke Taiwan disebut akan menghancurkan
perdagangan dunia. "Jarak tidak akan memberikan perlindungan terhadap pukulan
dahsyat yang tak terelakkan terhadap ekonomi global," kata Menteri Luar Negeri
Inggris, James Cleverly. Dia memperingatkan dalam pidato tentang hubungan
Inggris dengan Beijing. Baca juga:Perubahan Kebijakan Twitter Picu Lonjakan
Propaganda oleh Rusia dan China Sambutan ini berbeda dari apa yang
disampaikan Presiden Perancis Emmanuel Macron untuk menjauhkan Eropa dari
potensi keterlibatan AS dalam konflik di masa depan atas Taiwan. Macron dengan
tegas mendukung keberlanjutan keterlibatan yang dijaga dengan Beijing. Tapi,
dilansir dari  , Cleverly mengatakan bahwa tidak ada negara yang
dapat melindungi diri dari dampak perang di Taiwan. Dia menambahkan bahwa dia
ngeri memikirkan kehancuran finansial dan manusia yang akan terjadi. Mendesak
tidak ada pihak yang mengambil tindakan sepihak untuk mengubah status quo, dia
menegaskan relevansi Taiwan

In [16]:
# use function to simplify process
def remove_html(html_text):
    
    # define regex pattern
    pattern = r"<p>(.*?)</p>"
    
    # extract string between <p> and </p> tags, and join result to a sentence
    find = re.findall(pattern, html_text)
    text_ = ' '.join(find[:])
    
    # use html2text to clean html format
    text_maker = html2text.HTML2Text()
    text_maker.ignore_links = True
    text_maker.bypass_tables = True
    html_text = text_maker.handle(text_)
    
    # remove text enclosed in square brackets, asterisks or underscores
    temp_result = re.sub(r'\[.*?\]', '', html_text)
    result  = re.sub(r'(\*|_).*?(\*|_)', '', temp_result).replace(" \- ", " ")
    
    # return result
    return result

In [17]:
# loop through all df to clean html format
cleaned_article=[]
for i in df["news_article_raw"]:
    cleaned_article.append(remove_html(i))

In [18]:
# add new column to df to store cleaned html article
df["news_article_html_clean"] = cleaned_article

In [19]:
# function to preprocess text
# clean text from any symbols, punctuation, etc and casefolding
def text_casefold(text):
    text = re.sub(r", KOMPAS.com", "", text) # remove Kompas.com words
    text = re.sub(r"Baca", "", text) # remove Baca words
    text = re.sub(r"Baca Juga:", "", text) # remove Baca Juga words
    text = re.sub(r"#[A-Za-z0-9]+", "", text) # remove hashtag
    text = re.sub(r"@[A-Za-z0-9]+", "", text) # remove @
    text = re.sub(r"http\S+", "", text) # remove links
    text = re.sub(r"[0-9]+", "", text) # remove numbers    
    text = text.replace("\n", " ") # replace new line into space
    text = text.translate(str.maketrans("", "", string.punctuation)) # remove all punctuations
    text = text.strip(" ") # remove characters space from both left and right text
    text = text.lower() 
    return text    

# splitting string in the text into a list of tokens
def text_tokenize(text):
    text = word_tokenize(text) 
    return text

# removing stopwords in a text using nltk
def text_stopwords(text): 
    listStopwords = set(stopwords.words("indonesian"))
    filtered = []
    for word in text:
        if word not in listStopwords:
            filtered.append(word)
    text = filtered 
    return text

# reducing a word to its word stem using sastrawi, and convert to sentences
def text_stemming(text): 
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    list_word = [stemmer.stem(word) for word in text]
    sentence = " ".join(word for word in list_word)
    return sentence

In [20]:
# preview text (cleaning and casefold)
text_sample = text_casefold(df["news_article_html_clean"][0])
print(text_sample)

london invasi china ke taiwan disebut akan menghancurkan perdagangan dunia jarak tidak akan memberikan perlindungan terhadap pukulan dahsyat yang tak terelakkan terhadap ekonomi global kata menteri luar negeri inggris james cleverly dia memperingatkan dalam pidato tentang hubungan inggris dengan beijing  jugaperubahan kebijakan twitter picu lonjakan propaganda oleh rusia dan china sambutan ini berbeda dari apa yang disampaikan presiden perancis emmanuel macron untuk menjauhkan eropa dari potensi keterlibatan as dalam konflik di masa depan atas taiwan macron dengan tegas mendukung keberlanjutan keterlibatan yang dijaga dengan beijing tapi dilansir dari   cleverly mengatakan bahwa tidak ada negara yang dapat melindungi diri dari dampak perang di taiwan dia menambahkan bahwa dia ngeri memikirkan kehancuran finansial dan manusia yang akan terjadi mendesak tidak ada pihak yang mengambil tindakan sepihak untuk mengubah status quo dia menegaskan relevansi taiwan dengan kepentingan inggris “se

In [21]:
# preview text (tokenized text)
text_sample_token = text_tokenize(text_sample)
print(text_sample_token)

['london', 'invasi', 'china', 'ke', 'taiwan', 'disebut', 'akan', 'menghancurkan', 'perdagangan', 'dunia', 'jarak', 'tidak', 'akan', 'memberikan', 'perlindungan', 'terhadap', 'pukulan', 'dahsyat', 'yang', 'tak', 'terelakkan', 'terhadap', 'ekonomi', 'global', 'kata', 'menteri', 'luar', 'negeri', 'inggris', 'james', 'cleverly', 'dia', 'memperingatkan', 'dalam', 'pidato', 'tentang', 'hubungan', 'inggris', 'dengan', 'beijing', 'jugaperubahan', 'kebijakan', 'twitter', 'picu', 'lonjakan', 'propaganda', 'oleh', 'rusia', 'dan', 'china', 'sambutan', 'ini', 'berbeda', 'dari', 'apa', 'yang', 'disampaikan', 'presiden', 'perancis', 'emmanuel', 'macron', 'untuk', 'menjauhkan', 'eropa', 'dari', 'potensi', 'keterlibatan', 'as', 'dalam', 'konflik', 'di', 'masa', 'depan', 'atas', 'taiwan', 'macron', 'dengan', 'tegas', 'mendukung', 'keberlanjutan', 'keterlibatan', 'yang', 'dijaga', 'dengan', 'beijing', 'tapi', 'dilansir', 'dari', 'cleverly', 'mengatakan', 'bahwa', 'tidak', 'ada', 'negara', 'yang', 'dapat'

In [22]:
# preview text (removed stopwords)
text_sample_stop = text_stopwords(text_sample_token)
print(text_sample_stop)

['london', 'invasi', 'china', 'taiwan', 'menghancurkan', 'perdagangan', 'dunia', 'jarak', 'perlindungan', 'pukulan', 'dahsyat', 'terelakkan', 'ekonomi', 'global', 'menteri', 'negeri', 'inggris', 'james', 'cleverly', 'memperingatkan', 'pidato', 'hubungan', 'inggris', 'beijing', 'jugaperubahan', 'kebijakan', 'twitter', 'picu', 'lonjakan', 'propaganda', 'rusia', 'china', 'sambutan', 'berbeda', 'presiden', 'perancis', 'emmanuel', 'macron', 'menjauhkan', 'eropa', 'potensi', 'keterlibatan', 'as', 'konflik', 'taiwan', 'macron', 'mendukung', 'keberlanjutan', 'keterlibatan', 'dijaga', 'beijing', 'dilansir', 'cleverly', 'negara', 'melindungi', 'dampak', 'perang', 'taiwan', 'ngeri', 'memikirkan', 'kehancuran', 'finansial', 'manusia', 'mendesak', 'mengambil', 'tindakan', 'mengubah', 'status', 'quo', 'relevansi', 'taiwan', 'kepentingan', 'inggris', '“', 'kapal', 'kontainer', 'dunia', 'melewati', 'perairan', 'vital', 'selat', 'taiwan', 'sarat', 'barangbarang', 'eropa', 'penjuru', 'dunia', 'taiwan', 

In [23]:
# preview text (stemming)
text_sample_stem = text_stemming(text_sample_stop)
print(text_sample_stem)

london invasi china taiwan hancur dagang dunia jarak lindung pukul dahsyat elak ekonomi global menteri negeri inggris james cleverly ingat pidato hubung inggris beijing jugaperubahan bijak twitter picu lonjak propaganda rusia china sambut beda presiden perancis emmanuel macron jauh eropa potensi libat as konflik taiwan macron dukung lanjut libat jaga beijing lansir cleverly negara lindung dampak perang taiwan ngeri pikir hancur finansial manusia desak ambil tindak ubah status quo relevansi taiwan penting inggris  kapal kontainer dunia lewat air vital selat taiwan sarat barangbarang eropa penjuru dunia taiwan negara demokrasi kembang mata rantai rantai pasok global semikonduktor tingkat jugapernyataan dubes china bikin heboh eropa klarifikasi china aku wakil tradisi otoriter kejam tentang tradisi inggris  milik wajib generasi libat gagal tugas tahan bentuk tatanan internasional jugachinasingapura gelar latih militer armada kerah abai tantang tanda kuat lemah


In [24]:
# start loop through df to tokenize text
news_token = []
for i in df["news_article_html_clean"]:
    c_temp = text_casefold(i)
    c_temp = text_tokenize(c_temp)
    c_temp = text_stopwords(c_temp)
    news_token.append(c_temp)

In [25]:
# add new column to df to store tokenized article
df["news_article_token"] = news_token

In [26]:
# start loop through df to stem tokenized article
news_stem = []
for i in tqdm(df["news_article_token"]):
    s_temp = text_stemming(i)
    news_stem.append(s_temp)

100%|██████████| 415/415 [36:55<00:00,  5.34s/it]


In [27]:
# add new column to df to store stemmed article
df["news_article_stem"] = news_stem

In [28]:
# filter relevant columns (drop raw article)
res = df.drop(columns = ["publish_date", "news_article_raw", "news_article_semi"])

In [29]:
# show result
res.head()

,author,publish_date_adj,news_article_html_clean,news_article_token,news_article_stem
0,Tito Hilmawan Reditya,2023-04-26 18:15:00,"LONDON, KOMPAS.com Invasi China ke Taiwan dise...","[london, invasi, china, taiwan, menghancurkan,...",london invasi china taiwan hancur dagang dunia...
1,Aditya Jaya Iswara,2023-04-26 17:35:00,"BANGKOK, KOMPAS.com Polisi Thailand pada Selas...","[bangkok, polisi, thailand, selasa, menangkap,...",bangkok polisi thailand selasa tangkap istri p...
2,Danur Lambang Pristiandaru,2023-04-26 16:01:00,"PRETORIA, KOMPAS.com – Presiden Afrika Selatan...","[pretoria, –, presiden, afrika, selatan, cyril...",pretoria presiden afrika selatan cyril ramaph...
3,Aditya Jaya Iswara,2023-04-26 15:34:00,"WELLINGTON, KOMPAS.com Mantan Perdana Menteri ...","[wellington, mantan, perdana, menteri, selandi...",wellington mantan perdana menteri selandia jac...
4,Aditya Jaya Iswara,2023-04-26 14:48:00,"MOSKWA, KOMPAS.com Rusia mengganti KFC yang ke...","[moskwa, rusia, mengganti, kfc, invasi, negara...",moskwa rusia ganti kfc invasi negara ukraina p...


In [30]:
# pass to csv file
res.to_csv("data/kompas_news_clean.csv", index=0)

> The data has been successfully exported, therefore it is time to move to the next section in other notebook.